In [ ]:
import pandas as pd
import numpy as np

In [ ]:
annotation = pd.read_csv("../input/blood-cell-detection-dataset/annotations.csv")
annotation.head()

In [ ]:
images = annotation.image.unique()

In [ ]:
train_images = images[:70]
test_images = images[70:]

In [ ]:
import os
# working_dir = '/kaggle/working'
# train_dir = os.path.join( working_dir, 'train')
# test_dir = os.path.join( working_dir, 'test')

In [ ]:
yolo_train_dir = "/kaggle/working/yolo/train"
yolo_test_dir = "/kaggle/working/yolo/test"

for dd in [yolo_train_dir, yolo_test_dir]:
    for ss in ["images", "labels"]:
        print(os.path.join(dd, ss))
        os.makedirs(os.path.join(dd, ss), exist_ok=True)

In [ ]:
import glob
from tqdm import tqdm
import os
import cv2
import matplotlib.pyplot as plt


In [ ]:
image = cv2.imread('../input/blood-cell-detection-dataset/images/image-1.png')
image = image[:,:,2::-1]
print(image.shape)
plt.imshow(image)

In [ ]:
WIDTH = 256
HEIGHT = 256
cells_id = {
    'rbc': 0,
    'wbc': 1
}

In [ ]:
cells_classes = list(cells_id.keys())
cells_classes

In [ ]:
for image in train_images:
    lables_file = os.path.join(yolo_train_dir, 'labels', image.replace(".png", ".txt"))
    with open(lables_file, "w") as wobj:
        for box in annotation.loc[annotation.image == image].values:
            wobj.write("%d %f %f %f %f \n" % (
                cells_id[box[5]],
                ((box[3]+box[1])/2.0) / WIDTH,
                ((box[4]+box[2])/2.0) / HEIGHT,
                (box[3]-box[1]) / WIDTH,
                (box[4]-box[2]) / HEIGHT
            ))
        
for image in test_images:
    lables_file = os.path.join(yolo_test_dir, 'labels', image.replace(".png", ".txt"))
    with open(lables_file, "w") as wobj:
        for box in annotation.loc[annotation.image == image].values:
            wobj.write("%d %f %f %f %f \n" % (
                cells_id[box[5]],
                ((box[3]+box[1])/2.0) / WIDTH,
                ((box[4]+box[2])/2.0) / HEIGHT,
                (box[3]-box[1]) / WIDTH,
                (box[4]-box[2]) / HEIGHT
            ))

In [ ]:
SIZE_W = 640
SIZE_H = 640

In [ ]:
SIZE = (SIZE_W, SIZE_H)

def replace_image(src_file, dst_file, size=SIZE):
    image = cv2.imread(src_file) 
    image = cv2.resize(image, size)
    cv2.imwrite(dst_file, image)

In [ ]:
train_images_dir = "../input/blood-cell-detection-dataset/images"

for image in train_images:
    src_file = os.path.join(train_images_dir , image)
    dst_file = os.path.join(yolo_train_dir, 'images', image)
    replace_image(src_file, dst_file)
        
for image in test_images:
    src_file = os.path.join(train_images_dir , image)
    dst_file = os.path.join(yolo_test_dir, 'images', image)
    replace_image(src_file, dst_file)


In [ ]:
def draw_image(image_file, label_file, class_names):   
    image = cv2.imread(image_file)
        
    with open(label_file) as fobj:
        while True:            
            item = fobj.readline()
            if item is None or len(item)<=0:
                break
                
            item = item.split()
            
            lb = int(item[0])
            xc = float(item[1]) * SIZE_W
            yc = float(item[2]) * SIZE_H
            w = float(item[3]) * SIZE_W
            h = float(item[4]) * SIZE_H
        
            image = cv2.rectangle(image, (int(xc - w/2), int(yc - h/2)), (int(xc + w/2), int(yc + h/2)), (0,0,255), 1)
            image = cv2.putText(image, class_names[lb], (int(xc - w/2), int(yc - h/2 - 5)), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 255, 255), 1)
        
    return image

train_file = train_images[0]
image_file = os.path.join(yolo_train_dir, 'images', train_file)
label_file = os.path.join(yolo_train_dir, 'labels', train_file.replace(".png", ".txt"))

print(image_file)
print(label_file)

image = draw_image(image_file, label_file, cells_classes)
image = image[:,:,2::-1]
plt.imshow(image);

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt
%cd -

In [ ]:
!cat yolov5/data/coco128.yaml

In [ ]:
yaml_file="yolov5/data/blood_cell.yaml"
train_images_dir = os.path.join("..", yolo_train_dir, "images")
val_images_dir = os.path.join("..", yolo_test_dir, "images")


names_str=""
for item in cells_classes:
    names_str=names_str + ", \'%s\'"%item
names_str= "names: ["+names_str[1:]+"]"

with open(yaml_file, "w") as wobj:
    wobj.write("train: %s\n"%train_images_dir)
    wobj.write("val: %s\n"%val_images_dir)
    wobj.write("nc: %d\n"%len(cells_classes))
    wobj.write(names_str+"\n")

In [ ]:
import wandb
wandb.login()

In [ ]:
%cd yolov5
!bash weights/download_weights.sh

In [ ]:
!mv *.pt weights
!ls weights

In [ ]:
!python train.py --data data/blood_cell.yaml --batch-size 4 --epochs 300 --img-size 640 --project runs/train --name blood_cell --weights weights/yolov5s.pt --device 0

In [ ]:
!python detect.py --weights runs/train/blood_cell/weights/best.pt --source /kaggle/input/blood-cell-detection-dataset/images/image-1.png --name blood_cell --project runs/detect

In [ ]:
!mkdir -p tmp
!cp runs/train/blood_cell/test_batch0_labels.jpg tmp/0.jpg
!cp runs/train/blood_cell/test_batch0_pred.jpg tmp/1.jpg
!ffmpeg -r 1 -i tmp/%d.jpg -r 1 -y tmp/out.gif

In [ ]:
!ls -l runs/detect/blood_cell

In [ ]:
img = cv2.imread("runs/detect/blood_cell/image-1.png")
plt.imshow(img[:,:,2::-1]);

In [ ]:
import torch

model = torch.hub.load('/kaggle/working/yolov5', 'custom', path='runs/train/blood_cell/weights/best.pt', source='local')
# Images
imgs = ['/kaggle/input/blood-cell-detection-dataset/images/image-1.png']  # batch of images

# Inference
results = model(imgs)

# Results
results.print()
results.save()  # or .show()

results.xyxy[0]  # img1 predictions (tensor)
results.pandas().xyxy[0] 